I was able to get improvement in inference speed by using numpy instead of pandas

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
import gc
import riiideducation
env = riiideducation.make_env()
iter_tester = env.iter_test()

In [ ]:
results_c = pd.read_parquet('../input/riiid-data-prparation/results_question.parq')
results_u = pd.read_parquet('../input/riiid-data-prparation/results_user.parq')

In [ ]:
import lightgbm as lgbm

In [ ]:
model = lgbm.Booster(model_file='../input/riiid-training/booster_075291.txt')

In [ ]:
col1 = ['answered_correctly_content', 'content_id']
col2 = ['answered_correctly_user', 'questions_attempted', 'user_id']
rem =  ['prior_question_elapsed_time'] 
pk = ['user_id','content_id']

In [ ]:
used_cols = rem+col1+col2
init_cols = pk+rem

In [ ]:
# required only for numpy 
results_u = results_u.values
results_c = results_c.values

inference using numpy

In [ ]:
print('time on each step:')
t = time.time()
for test_df, sample_prediction_df in iter_tester:
    test_df = test_df.reset_index()
    test_df['answered_correctly'] = 0.6571448171182185
    nump = test_df[init_cols].values
    for i in range(len(test_df)):
        uinf = results_u[np.where(nump[i,0]==results_u[:,2])[0]]
        qinf = results_c[np.where(nump[i,1]==results_c[:,1])[0]]
        if len(uinf)==0 or len(qinf)==0:
            continue
        test_df.loc[i,'answered_correctly'] = model.predict(np.hstack((nump[i,2].reshape(1,-1),qinf,uinf)))
    env.predict(test_df.loc[test_df['content_type_id']==0,['row_id','answered_correctly']])
    print(time.time()-t)
    t = time.time()

Inference using pandas

In [ ]:
print('time on each step:')
t = time.time()
for test_df, sample_prediction_df in iter_tester:
    test_df = test_df.reset_index()
    test_df = test_df.merge(results_c,on='content_id',right_index=True)
    temp = test_df.merge(results_u,on='user_id',right_index=True)
    test_df['answered_correctly'] = 0.6571448171182185
    test_df.loc[temp.index.to_list(),'answered_correctly'] = model.predict(temp[used_cols])
    env.predict(test_df.loc[test_df['content_type_id']==0,['row_id','answered_correctly']])
    print(time.time()-t)
    t = time.time()